In [1]:
from bs4 import BeautifulSoup as bs
import selenium
import requests 
import pandas as pd
import datetime

In [3]:
with open("spreads.html") as fp:
    soup = bs(fp, "html.parser")

In [10]:
spreads_results = pd.read_html("spreads.html")[0]

In [12]:
spreads_results.to_csv("spreads.csv")

In [2]:
from tda import auth, client
import json
 
token_path = R'.\token.pickle'
api_key = 'DRSMU4TL964FO3QNBQHVL78X9SUPGGIL@AMER.OAUTHAP'
redirect_uri = "http://localhost"
try:
    c = auth.client_from_token_file(token_path, api_key)
except FileNotFoundError:
    from selenium import webdriver
    with webdriver.Chrome() as driver:
        c = auth.client_from_login_flow(
            driver, api_key, redirect_uri, token_path)

In [3]:
para = c.Options

chains = c.get_option_chain(symbol="SPY", contract_type=para.ContractType.PUT, to_date=datetime.date(2021,5,28))

In [31]:
initial = pd.json_normalize(chains.json(), record_path="monthlyStrategyList")

In [4]:
initial = pd.json_normalize(chains.json())
spreads = pd.json_normalize(initial["putExpDateMap.2021-05-28:7.475.0"].explode())

In [5]:
initial

,symbol,status,underlying,strategy,interval,isDelayed,isIndex,interestRate,underlyingPrice,volatility,...,putExpDateMap.2021-05-28:7.475.0,putExpDateMap.2021-05-28:7.480.0,putExpDateMap.2021-05-28:7.485.0,putExpDateMap.2021-05-28:7.490.0,putExpDateMap.2021-05-28:7.495.0,putExpDateMap.2021-05-28:7.500.0,putExpDateMap.2021-05-28:7.510.0,putExpDateMap.2021-05-28:7.520.0,putExpDateMap.2021-05-28:7.530.0,putExpDateMap.2021-05-28:7.540.0
0,SPY,SUCCESS,None,SINGLE,0.0,True,False,0.1,417.005,29.0,...,"[{'putCall': 'PUT', 'symbol': 'SPY_052821P475'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P480'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P485'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P490'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P495'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P500'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P510'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P520'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P530'...","[{'putCall': 'PUT', 'symbol': 'SPY_052821P540'..."


In [7]:
explode = []

for i in initial.columns:
    try: 
        explode.append(pd.json_normalize(initial[i].explode()))
    except:
        pass

In [9]:
complete = pd.concat(explode)

In [53]:
complete.loc[(complete["delta"] > -0.070) & (complete["delta"] < -0.029)][["symbol", "daysToExpiration","delta","mark"]]

,symbol,daysToExpiration,delta,mark
0,SPY_052121P411,0,-0.034,0.05
0,SPY_052121P412,0,-0.056,0.08
0,SPY_052421P402,3,-0.032,0.11
0,SPY_052421P403,3,-0.038,0.13
0,SPY_052421P404,3,-0.043,0.14
0,SPY_052421P405,3,-0.050,0.16
0,SPY_052421P406,3,-0.059,0.19
0,SPY_052621P393,5,-0.033,0.18
0,SPY_052621P394,5,-0.035,0.19
0,SPY_052621P395,5,-0.039,0.21
